<font color='darkorange'> Unless otherwise noted, **this notebook will not be reviewed or autograded.**</font> You are welcome to use it for scratchwork, but **only the files listed in the exercises will be checked.**

---

# Exercises

For these exercises, add your functions to the *apputil\.py* file and *app\.py* file as instructed. *These exercises use the same [Titanic dataset](https://www.kaggle.com/competitions/titanic/data) as the lab.*


## Exercise 1: Survival Patterns


For this exercise you will analyze survival patterns on the Titanic by looking at passenger class, sex, and age group. Name the function `survival_demographics()`.

1. Create a new column in the Titanic dataset that classifies passengers into age categories (i.e., a pandas `category` series). The categories should be:
    - Child (up to 12)
    - Teen (13–19)
    - Adult (20–59)
    - Senior (60+)  
  
	Hint: The `pd.cut()` function might come in handy here.

2. Group the passengers by class, sex, and age group.  

3. For each group, calculate:  
    - The total number of passengers, `n_passengers`
    - The number of survivors, `n_survivors`
    - The survival rate, `survival_rate`

4. Return a table that includes the results for *all* combinations of class, sex, and age group.  

5. Order the results so they are easy to interpret.  

6. Come up with a clear question that your results table makes you curious about (e.g., “Did women in first class have a higher survival rate than men in other classes?”). Write this question in your `app.py` file above the call to your visualization function, using `st.write("Your Question Here")`.
   
7. Create a Plotly visualization in a function named `visualize_demographic()` that directly addresses your question by returning a Plotly figure (e.g., `fig = px. ...`). You are free to choose the chart type that you think best communicates the findings. Be creative — try different approaches, compare them, and ensure that your chart clearly answers the question you posed.


In [8]:
import pandas as pd

# Read in titanic dataset
df = pd.read_csv('https://raw.githubusercontent.com/leontoddjohnson/datasets/main/data/titanic.csv')

# Use pd.cut to create age groups based on the Age column
df['Age_Group'] = pd.cut(df["Age"], bins = [0, 12, 20, 60, 120], labels = ["child", "teen", "adult", "senior"])

# Create a column to allow for easy summation of population
df['Count'] = 1


In [4]:
# Group the passangers by class, sex, and group
df_grouped = df.groupby(['Pclass', 'Sex', 'Age_Group']).agg({
    'Survived': 'sum',
    'Count': 'sum'
}).reset_index()

# Calculate survival percentage
df_grouped['Survival_Percentage'] = ((df_grouped['Survived'] / df_grouped['Count']) * 100).round(2)

df_grouped.head()


C:\Users\chris\AppData\Local\Temp\ipykernel_46864\1154874564.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_grouped = df.groupby(['Pclass', 'Sex', 'Age_Group']).agg({


,Pclass,Sex,Age_Group,Survived,Count,Survival_Percentage
0,1,female,child,0,1,0.0
1,1,female,teen,13,13,100.0
2,1,female,adult,67,69,97.1
3,1,female,senior,2,2,100.0
4,1,male,child,3,3,100.0


In [4]:
import streamlit as st
import plotly.express as px

# Use the grouped dataframe for analysis
df = df_grouped

# Create the two categories for visualization analysis.
df['Category'] = "Other"
df.loc[(df['Pclass'] == 1) & (df['Sex'] == 'male'), 'Category'] = '1st Class Male'
df.loc[(df['Pclass'] == 3) & (df['Age_Group'] == 'child'), 'Category'] = '3rd Class Child'

# Filter the Dataframe for the visualization
df_viz = df[df['Category'] != "Other"]

df_viz


,Pclass,Sex,Age_Group,Survived,Count,Survival_Percentage,Category
4,1,male,child,3,3,100.00,1st Class Male
5,1,male,teen,1,4,25.00,1st Class Male
6,1,male,adult,35,82,42.68,1st Class Male
7,1,male,senior,1,12,8.33,1st Class Male
16,3,female,child,11,23,47.83,3rd Class Child
20,3,male,child,9,25,36.00,3rd Class Child


In [5]:
fig = px.histogram(
            df_viz,  
             y='Category',
             x='Survival_Percentage',
             histfunc='avg',
             hover_data=['Category'],
             template='plotly_white',
             color_discrete_sequence=px.colors.qualitative.D3
            )
fig.show()

In [6]:
import streamlit as st

from apputil import *

st.write(
'''
# Did men in the first class have a higher survival rate than children in the third class?

'''
)
# Generate and display the figure
fig1 = visualize_demographic()
st.plotly_chart(fig1, use_container_width=True)

2025-11-12 21:42:53.633 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-12 21:42:53.862 
  command:

    streamlit run c:\Users\chris\miniconda3\envs\week-5\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-11-12 21:42:53.863 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-12 21:42:53.863 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
c:\Users\chris\OneDrive - Indiana University\Graduate School\MIS\INFO-H 501\Projects\week-5\apputil.py:15: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

2025-11-12 21:42:54.180 Thread 'MainThread': missing ScriptRunConte

DeltaGenerator()

## Exercise 2: Family Size and Wealth

Using the Titanic dataset, write a function named `family_groups()` to explore the relationship between family size, passenger class, and ticket fare.  

1. Create a new column in the Titanic dataset that represents the total family size for each passenger, `family_size`. Family size is defined as the number of siblings/spouses aboard plus the number of parents/children aboard, plus the passenger themselves.

2. Group the passengers by family size and passenger class. For each group, calculate:  
   - The total number of passengers, `n_passengers`
   - The average ticket fare, `avg_fare`
   - The minimum and maximum ticket fares (to capture variation in wealth), `min_fare` and `max_fare`

3. Return a table with these results, sorted so that the values are clear and easy to interpret (for example, by class and then family size).

4. Write a function called `last_names()` that extracts the last name of each passenger from the `Name` column, and returns the count for each last name (i.e., a pandas series with last name as index, and count as value). Does this result agree with that of the data table above? Share your findings in your app using `st.write`.

5. Just like you did in Exercise 1, come up with a clear question that your results makes you curious about. Write this question in your app.py file above the call to your visualization function. Then, create a Plotly visualization in a function named `visualize_families()` that directly addresses your question. As in Exercise 1 you are free to choose the chart type that you think best communicates the findings.

In [30]:
# Read in titanic dataset
df = pd.read_csv('https://raw.githubusercontent.com/leontoddjohnson/datasets/main/data/titanic.csv')

df['family_size'] = df['SibSp'] + df['Parch'] + 1
df['Count'] = 1

df_histo = df.copy()

df_grouped = df.groupby(['Pclass','family_size']).agg(
    n_passengers=('Count', 'sum'),
    avg_fare=('Fare', 'mean'),
    min_fare=('Fare', 'min'),
    max_fare=('Fare', 'max')
).reset_index()

df_grouped

,Pclass,family_size,n_passengers,avg_fare,min_fare,max_fare
0,1,1,109,63.672514,0.0000,512.3292
1,1,2,70,91.848039,29.7000,512.3292
2,1,3,24,95.681075,26.2833,211.5000
3,1,4,7,133.521429,120.0000,151.5500
4,1,5,2,262.375000,262.3750,262.3750
5,1,6,4,263.000000,263.0000,263.0000
6,2,1,104,14.066106,0.0000,73.5000
7,2,2,34,24.682962,11.5000,33.0000
8,2,3,31,31.693819,13.0000,73.5000
9,2,4,13,36.575969,11.5000,65.0000


In [32]:
fig = px.histogram(df_histo, 
                   x = 'family_size',
                   template='plotly_white',
                   color_discrete_sequence=px.colors.qualitative.D3
                  )

fig.update_layout(
    bargap=0.1, # gap between bars of adjacent location coordinates
)

fig.show()

In [15]:
df = pd.read_csv('https://raw.githubusercontent.com/leontoddjohnson/datasets/main/data/titanic.csv')

df['last_name'] = df['Name'].apply(lambda x: x.split(',')[0].strip())
df['Count'] = 1

df_groupby = df.groupby('last_name').agg(total_count=('Count', 'sum'))

df_groupby

,total_count
last_name,
Abbing,1
Abbott,2
Abelson,2
Adahl,1
Adams,1
...,...
de Mulder,1
de Pelsmaeker,1
del Carlo,1


In [21]:
fig = px.histogram(df_groupby, 
                   x='total_count', 
                   template='plotly_white',
                   color_discrete_sequence=px.colors.qualitative.D3
                  )

fig.update_layout(
    bargap=0.1, # gap between bars of adjacent location coordinates
)

fig.show()

## Bonus Question

Add a new column, `older_passenger`, to the Titanic dataset that indicates whether each passenger’s age is above the median age for *their* passenger class. So, suppose row $x$ is in passenger class 2. Then, a value of `True` at row $x$ would indicate that passenger older than 50% of class 2 passengers, and `False` would indicate that they younger.

- You should use pandas functions to accomplish this.
- The new column should contain Boolean values (True if the age is above the median, False if less than or equal to).
- Return the updated table in the function `determine_age_division()`

Once you’ve created this column, consider how this age division relates to your analysis above. Try to visualize this analysis in Plotly using the function name `visualize_age_division()`.